In [1]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 
from nltk.corpus import stopwords
morph = MorphAnalyzer()

C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import warnings
import numpy as np

## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

## Данные

In [3]:
def remove_tags(texts):
    return [re.sub(r'{[*]}+>', '', text) for text in texts]
#В очистке мне интересно было посмотреть, как он разобъёт текст на темы, не имея подсказок в виде терминологии на английском
#Прошу прощения за длинный и топорный код по склейкам-очисткам
def clean(words):
    stp = stopwords.words('russian')
    words = [word.lower() for word in words]
    lst = []
    wrds = [re.findall('[а-я]+', word) for word in words]
    for wrd in wrds:
        lst2 = []
        for wd in wrd:
            if wd not in stp:
                lst2.append(wd)
        xx = ' '.join(lst2)
        lst.append(xx)
    clean = [morph.parse(word)[0].normal_form for word in lst if len(set(word)) > 1]
    return clean

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [4]:
texts = open('habr_texts.txt', encoding = 'utf-8').read().splitlines()

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [5]:
texts = [clean(word_tokenize(text.lower())) for text in texts]

In [6]:
print(len(texts))

4121


In [7]:
from collections import Counter

In [8]:
allwords = []
for x in texts:
    for i in x:
        allwords.append(i)
print(len(allwords))

2962789


In [9]:
x = Counter(allwords).most_common()
print(x[:50])

[('это', 36098), ('который', 33128), ('один', 17493), ('дать', 16024), ('такой', 12884), ('год', 12621), ('время', 12051), ('использовать', 11970), ('мочь', 11509), ('работа', 11410), ('система', 10929), ('свой', 10128), ('работать', 9632), ('самый', 9428), ('наш', 8940), ('компания', 8680), ('другой', 8484), ('человек', 8417), ('код', 8253), ('каждый', 8235), ('проект', 7749), ('пользователь', 7654), ('очень', 7452), ('случай', 7271), ('быть', 7208), ('должный', 7087), ('нужно', 7064), ('также', 7062), ('приложение', 7006), ('сделать', 6739), ('например', 6724), ('тот', 6349), ('новый', 6306), ('несколько', 6258), ('функция', 6211), ('файл', 6079), ('возможность', 6060), ('проблема', 6056), ('просто', 6044), ('два', 6038), ('иметь', 6034), ('решение', 5983), ('делать', 5904), ('задача', 5902), ('получить', 5792), ('какой', 5784), ('стать', 5666), ('результат', 5656), ('игра', 5424), ('устройство', 5414)]


Если модель будет совсем плохо работать, попробуем добавить эти слова и сравнить качество.

In [10]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [11]:
dictinary = gensim.corpora.Dictionary(texts)

In [12]:
dictinary.filter_extremes(no_above=0.3, no_below=30)
dictinary.compactify()

In [13]:
print(dictinary)

Dictionary(6408 unique tokens: ['распредел нный', 'клуб', 'глубоко', 'дизайн', 'каскад']...)


Преобразуем наши тексты в мешки слов. 

In [14]:
corpus = [dictinary.doc2bow(text) for text in texts]

In [15]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, passes=3)

C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [16]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    lda2 = gensim.models.LdaMulticore(corpus, 50, id2word=dictinary, passes=3)
    lda3 = gensim.models.LdaMulticore(corpus, 20, id2word=dictinary, passes=3)

In [17]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    print('Перплексия на 100 темах: ', lda.log_perplexity(corpus[:2000], total_docs=100))
    print('Перплексия на 50 темах: ', lda2.log_perplexity(corpus[:2000], total_docs=100))
    print('Перплексия на 20 темах: ', lda3.log_perplexity(corpus[:2000], total_docs=100))

Перплексия на 100 темах:  -46.17318780129595
Перплексия на 50 темах:  -26.942996425054794
Перплексия на 20 темах:  -15.138136292876991


Теперь посмотрим когерентность.

In [18]:
#with warnings.catch_warnings():
#    warnings.filterwarnings("ignore",category=DeprecationWarning)
print('Когерентность на 100 темах: ', gensim.models.CoherenceModel(model=lda, texts=texts, dictionary=dictinary, coherence='c_v').get_coherence())
print('Когерентность на 50 темах: ', gensim.models.CoherenceModel(model=lda2, texts=texts, dictionary=dictinary, coherence='c_v').get_coherence())
print('Когерентность на 20 темах: ', gensim.models.CoherenceModel(model=lda3, texts=texts, dictionary=dictinary, coherence='c_v').get_coherence())

Когерентность на 100 темах:  0.3787498612403591
Когерентность на 50 темах:  0.4172323433408818
Когерентность на 20 темах:  0.37048508507885325


Хотя лучшая перплексия на ста темах, лучшая когерентность на 50-ти. Как найти баланс? Попробуем еще 35 тем.

In [19]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    lda4 = gensim.models.LdaMulticore(corpus, 35, id2word=dictinary, passes=3)
    print('Перплексия на 35 темах: ', lda4.log_perplexity(corpus[:2000], total_docs=100))
    print('Когерентность на 35 темах: ', gensim.models.CoherenceModel(model=lda4, texts=texts, dictionary=dictinary, coherence='c_v').get_coherence())

Перплексия на 35 темах:  -21.063387398149874
Когерентность на 35 темах:  0.3956190116257626


Пожалуй, остановимся на этом как на сбалансированном варианте.

Посмотрим на топики.

In [21]:
lda4.print_topics()

[(4,
  '0.012*"элемент" + 0.010*"компонент" + 0.010*"класс" + 0.006*"метод" + 0.006*"тест" + 0.005*"список" + 0.005*"свойство" + 0.005*"библиотека" + 0.005*"строка" + 0.004*"значение"'),
 (20,
  '0.006*"тест" + 0.004*"диск" + 0.004*"операция" + 0.004*"устройство" + 0.004*"глаз" + 0.003*"тестирование" + 0.003*"лазер" + 0.003*"метод" + 0.002*"пациент" + 0.002*"лицо"'),
 (17,
  '0.006*"клиент" + 0.004*"сайт" + 0.004*"продукт" + 0.003*"ошибка" + 0.003*"атака" + 0.003*"сервис" + 0.003*"программа" + 0.003*"управление" + 0.002*"бизнес" + 0.002*"сообщение"'),
 (2,
  '0.013*"файл" + 0.010*"машина" + 0.009*"база" + 0.009*"резервный" + 0.007*"диск" + 0.006*"копирование" + 0.005*"устройство" + 0.005*"сервер" + 0.005*"копия" + 0.005*"виртуальный"'),
 (15,
  '0.005*"игра" + 0.004*"сайт" + 0.003*"программа" + 0.003*"элемент" + 0.003*"список" + 0.003*"доступ" + 0.003*"ещ" + 0.002*"деньга" + 0.002*"игрок" + 0.002*"обучение"'),
 (10,
  '0.007*"технология" + 0.006*"сеть" + 0.006*"устройство" + 0.004*"вир

<b>Первая</b> тема, которая мне показалась неплохой, явно про коммерческую разработку: '0.006*"клиент" + 0.004*"сайт" + 0.004*"продукт" + 0.003*"ошибка" + 0.003*"атака" + 0.003*"сервис" + 0.003*"программа" + 0.003*"управление" + 0.002*"бизнес" + 0.002*"сообщение"'.  
<b>Вторая</b> неплохо выглядящая тема - что-то про компьютерную безопасность: '0.007*"технология" + 0.006*"сеть" + 0.006*"устройство" + 0.004*"вирус" + 0.004*"производство" + 0.003*"интернет" + 0.003*"пациент" + 0.003*"безопасность" + 0.003*"сервис" + 0.003*"российский"'.  
<b>Третья</b> хорошая тема - тема про игры: '0.029*"игра" + 0.008*"игрок" + 0.004*"игровой" + 0.004*"ещ" + 0.003*"играть" + 0.002*"продукт" + 0.002*"рынок" + 0.002*"мой" + 0.002*"карта" + 0.002*"друг"'.

In [22]:
dictinary = gensim.corpora.Dictionary(texts)
dictinary.filter_extremes(no_above=0.3, no_below=30)
dictinary.compactify()
print(dictinary)

Dictionary(6408 unique tokens: ['распредел нный', 'клуб', 'глубоко', 'дизайн', 'каскад']...)


In [23]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary)
corpus = tfidf[corpus]

In [24]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    lda5 = gensim.models.LdaMulticore(corpus, 35, id2word=dictinary, passes=3) 

In [26]:
with warnings.catch_warnings():      
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    print('Перплексия на 35 темах: ', lda5.log_perplexity(corpus[:2000], total_docs=100))
    print('Когерентность на 35 темах: ', gensim.models.CoherenceModel(model=lda5, texts=texts, dictionary=dictinary, coherence='c_v').get_coherence())

Перплексия на 35 темах:  -131.76430775205964
Когерентность на 35 темах:  0.3011050895655429


Перплексия неожиданно стала очень низкой, зато когерентность сильно упала.

In [27]:
lda5.print_topics()

[(11,
  '0.002*"пост" + 0.002*"удалённый" + 0.002*"локализация" + 0.002*"безопасность" + 0.001*"преступник" + 0.001*"улица" + 0.001*"собственность" + 0.001*"анимация" + 0.001*"устройство" + 0.001*"язык"'),
 (22,
  '0.004*"метка" + 0.002*"уравнение" + 0.001*"формула" + 0.001*"клиент" + 0.001*"класс" + 0.001*"пакет" + 0.001*"файл" + 0.001*"тег" + 0.001*"тест" + 0.001*"диаграмма"'),
 (17,
  '0.004*"резервный" + 0.003*"копирование" + 0.003*"скрипт" + 0.002*"вселенная" + 0.002*"бот" + 0.002*"виртуальный" + 0.002*"сервер" + 0.002*"восстановление" + 0.002*"файл" + 0.002*"вм"'),
 (13,
  '0.004*"файл" + 0.002*"сервер" + 0.002*"пакет" + 0.002*"тест" + 0.002*"модуль" + 0.002*"параметр" + 0.002*"клиент" + 0.002*"устройство" + 0.002*"порт" + 0.002*"адрес"'),
 (23,
  '0.003*"атака" + 0.002*"нейросеть" + 0.002*"значение" + 0.002*"элемент" + 0.002*"объект" + 0.002*"игра" + 0.002*"координата" + 0.002*"событие" + 0.002*"метод" + 0.002*"ячейка"'),
 (30,
  '0.002*"пакет" + 0.001*"протокол" + 0.001*"транза

Забавно: хотя сохранились разные бизнес-темы и технологии, неожиданно совсем ушла тема про вирусы (точнее тема про безопасность несколько изменилась). В первой модели слово "элемент" явно играло большую роль: особенно хорошо это было в 4 теме, где явно описывались какие-то языки программирования или что-то подобное; во второй модели эта ярко выраженная тема вроде бы вовсе пропадает. Также пропадает и 20-я тема про тестирование, точнее расходится по другим темам; полностью пропадает 3 тема (про товар).

In [28]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [29]:
stexts = [' '.join(text) for text in texts]

In [30]:
print(len(stexts))

4121


In [33]:
vectorizer = TfidfVectorizer(max_features=35000, min_df=4, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

In [39]:
model1 = NMF(n_components=100)
model1.fit(X)
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(100):
        
        #топ n слов для темы.
        words_ids = model1.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);
get_nmf_topics(model1, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 90,Topic # 91,Topic # 92,Topic # 93,Topic # 94,Topic # 95,Topic # 96,Topic # 97,Topic # 98,Topic # 99
0,бизнес,продукт,игра,сервер,смартфон,класс,марс,память,атака,запрос,...,российский,анимация,видео,регистратор,символ,плагин,лазер,карта,сотрудник,регистр
1,заказчик,битрикс,игровой,настройка,дисплей,метод,космический,оперативный,безопасность,ответ,...,закон,кадр,пост,видео,строка,иконка,роговица,платёж,офис,адрес
2,ит,рынок,играть,выделить,экран,реализация,станция,гб,защита,выполнение,...,россия,движение,рассказывать,китайский,шрифт,веб,глаз,банка,рабочий,байт
3,управление,функционал,движок,адрес,ноутбук,вызов,аппарат,производительность,ботнет,кэш,...,рф,анимировать,комментарий,запись,текст,редактор,операция,банк,работник,битый
4,планирование,платформа,приставка,нагрузка,аккумулятор,интерфейс,луна,мб,злоумышленник,сессия,...,фас,персонаж,обсудить,рубль,буква,настройка,пациент,платёжный,руководитель,инструкция
5,технический,менеджер,жанр,конфигурация,гб,реализовать,наса,накопитель,вредоносный,строка,...,страна,экран,хабра,глонасс,дисплей,установка,хирург,оплата,менеджер,порт
6,исполнитель,продакт,инди,хостинг,планшет,экземпляр,миссия,потребление,угроза,параметр,...,роскомнадзор,слой,публиковать,видеорегистратор,таблица,инструмент,линза,банковский,корпоративный,флаг
7,архитектор,конкурент,персонаж,порт,клавиатура,свойство,космос,объ,хакер,субд,...,суд,нативный,удобно,модель,глиф,название,коррекция,шлюз,отпуск,таймер
8,услуга,тестирование,выпустить,веб,дюймовый,наследование,полёт,субд,информационный,получение,...,блокировка,спрайт,слушать,бренд,цифра,поиск,разрез,терминал,удалённый,значение
9,инфраструктура,идея,автомат,база,корпус,параметр,земля,поток,взлом,ошибка,...,государственный,эффект,делиться,матрица,кодировка,билд,лентикула,страна,зарплата,прерывание


Беглый взгляд видит те же темы, что и раньше - бизнес, продукты, игры, серверы, смартфоны, космос, память (в связи с серверами?), компбез, современные технологии и т.д. Вроде бы все слова более-мееее подходят друг к другу, но темы очень узкоспециализированные.

In [40]:
model2 = NMF(n_components=50)
model2.fit(X)
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(50):
        
        #топ n слов для темы.
        words_ids = model2.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);
get_nmf_topics(model2, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 41,Topic # 42,Topic # 43,Topic # 44,Topic # 45,Topic # 46,Topic # 47,Topic # 48,Topic # 49,Topic # 50
0,сотрудник,российский,игра,сервер,смартфон,класс,марс,виртуальный,атака,запрос,...,диск,текстура,продукт,компонент,сообщение,клиент,письмо,блок,процессор,пароль
1,программист,рынок,игрок,адрес,телефон,объект,земля,машина,безопасность,таблица,...,резервный,изображение,мобильный,компонента,сервис,номер,рассылка,транзакция,память,флаг
2,мой,россия,игровой,домен,дисплей,метод,луна,вм,защита,база,...,бэкап,карта,дизайн,состояние,отправка,звонок,адрес,контроллер,ядро,логин
3,деньга,рубль,играть,настройка,экран,вызов,аппарат,хост,вредоносный,бд,...,копия,анимация,рынок,библиотека,событие,вызов,подписчик,кнопка,производительность,авторизация
4,жизнь,страна,персонаж,выделить,аккумулятор,реализация,планета,реальность,злоумышленник,индекс,...,копирование,шейдёр,инструмент,свойство,канал,услуга,почта,ротация,регистр,учётный
5,опыт,налог,движок,хостинг,ноутбук,свойство,космический,виртуализация,ботнет,субд,...,восстановление,пиксель,платформа,рендеринг,отправлять,оператор,домен,датчик,гб,задание
6,бизнес,миллион,геймплей,нагрузка,гб,экземпляр,астероид,гипервизора,угроза,запись,...,запись,точка,дизайнер,событие,уведомление,бизнес,почтовый,плата,инструкция,телефон
7,ещ,доход,приставка,конфигурация,модель,интерфейс,станция,ос,хакер,строка,...,схд,объект,интерфейс,шаблон,байт,продажа,электронный,длина,архитектура,адрес
8,думать,закон,жанр,кластер,батарея,реализовать,солнечный,автомобиль,программа,ответ,...,хранение,координата,сервис,отображать,отправить,клиентский,отправитель,схема,видеокарта,доступ
9,идея,бизнес,инди,база,корпус,значение,орбита,сетевой,информационный,выполнение,...,база,свет,тестирование,отрисовка,очередь,интеграция,получатель,питание,драйвер,аутентификация


Часть тем сохраняется (космос, игры, продакшн, базы данных), часть уходит в другие, но слова вроде бы по-прежнему осмысленны друг с другом. Видимо, еще есть запас "ширины" для распределения текстов по темам.

In [41]:
model3 = NMF(n_components=35)
model3.fit(X)
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(35):
        
        #топ n слов для темы.
        words_ids = model3.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);
get_nmf_topics(model3, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30,Topic # 31,Topic # 32,Topic # 33,Topic # 34,Topic # 35
0,продукт,рынок,игра,файл,память,класс,космический,сервер,уязвимость,сайт,...,ключ,облачный,звук,товар,принтер,пакет,робот,камера,сеть,обучение
1,сотрудник,российский,игрок,папка,процессор,объект,спутник,виртуальный,атака,реклама,...,пароль,облако,сигнал,скидка,печать,репозиторий,автомобиль,смартфон,трафик,нейросеть
2,программист,страна,игровой,строка,диск,метод,орбита,машина,безопасность,страница,...,сообщение,инфраструктура,частота,магазин,станок,сборка,датчик,видео,интернет,изображение
3,бизнес,россия,играть,скрипт,ядро,значение,марс,настройка,злоумышленник,домен,...,шифрование,центр,наушник,цена,печатать,контейнер,машина,экран,адрес,нейронный
4,опыт,миллион,персонаж,директория,производительность,вызов,аппарат,база,вредоносный,контент,...,токен,дата,усилитель,распродажа,пластик,плагин,ребёнок,дисплей,связь,алгоритм
5,идея,рубль,движок,имя,гб,реализация,ракета,кластер,защита,рекламный,...,запись,сервис,звуковой,покупка,материал,установка,робототехника,регистратор,протокол,модель
6,мой,бизнес,геймплей,программа,накопитель,свойство,земля,домен,атаковать,хостинг,...,учётный,услуга,напряжение,покупатель,производство,скрипт,движение,телефон,сетевой,машинный
7,деньга,доход,мобильный,флаг,виртуальный,экземпляр,наса,адрес,обновление,блокировка,...,адрес,виртуальный,искажение,пятница,деталь,настройка,дрон,карта,оператор,сеть
8,думать,налог,приставка,запускать,запись,переменный,луна,конфигурация,ботнет,интернет,...,аккаунт,ит,музыка,заказ,модель,конфигурация,беспилотный,гб,канал,интеллект
9,понимать,деньга,жанр,путь,оперативный,параметр,станция,хост,уязвимый,посетитель,...,подпись,оборудование,ток,акция,изделие,запуск,реб,модель,маршрутизатор,слой


Темы до сих пор осмысленные (только что за реб в теме №32?), распределение примерно то же самое - интернет, базы, современные технологии, космос, сайты и т.д. Пожалуй, учитывая сохранение осмысленности распределения слов, это самое лучшее распределение по темам (т.к. нет слишком узкой специализации).